## Spectral Finance Take-Home Challenge
#### by Allen Wang

In [363]:
import requests
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [364]:
#This is the query we are going to be using to extract important borrower information including borrowing, liquidation, and repayment history
query = """{
  borrows(first: 100) {
    user{id,depositHistory {
      amount
    },borrowHistory {
      amount
    },liquidationCallHistory {collateralAmount}
    ,repayHistory{amountAfterFee}}
    
}
}"""

In [365]:
url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-multy-raw'
r = requests.post(url, json={'query': query})
print(r.status_code)
print(r.text)

200
{"data":{"borrows":[{"user":{"borrowHistory":[{"amount":"500000000000000000000"},{"amount":"2800000000000000000000"},{"amount":"807886931374901115857"},{"amount":"2000000000000000000000"},{"amount":"3000000000000000000"},{"amount":"2501334870977580000000"}],"depositHistory":[{"amount":"1845206409969912381402"},{"amount":"8000000000000000000"},{"amount":"1295000000000000000000"},{"amount":"7500000000000000000"},{"amount":"6000000000000000000"},{"amount":"6500000000000000000"},{"amount":"4000000000000000000"},{"amount":"6000000000000000000"}],"id":"0xfb5a2da7c365bfdd817bcc29a49c97b80e450f3d","liquidationCallHistory":[],"repayHistory":[{"amountAfterFee":"149000000000000000000"},{"amountAfterFee":"13838716775749340"},{"amountAfterFee":"7549392887426286090425"},{"amountAfterFee":"3001658611854892528"},{"amountAfterFee":"252136385398999946603"},{"amountAfterFee":"268382889195426708695"},{"amountAfterFee":"409145511749732737831"}]}},{"user":{"borrowHistory":[{"amount":"1000000000"},{"amou

In [366]:
json_data = json.loads(r.text)

In [367]:
df_data = json_data['data']['borrows']
df = pd.DataFrame(df_data)

In [368]:
#The data is not properly structured because of the complexity of the nature of the dataset. We want to parse the borrowing, repaying, and liquidation information into their respective column
df

,user
0,{'borrowHistory': [{'amount': '500000000000000...
1,"{'borrowHistory': [{'amount': '1000000000'}, {..."
2,{'borrowHistory': [{'amount': '900000000000000...
3,"{'borrowHistory': [{'amount': '80302898'}], 'd..."
4,{'borrowHistory': [{'amount': '208748722563150...
...,...
95,"{'borrowHistory': [{'amount': '400000000'}, {'..."
96,{'borrowHistory': [{'amount': '275890249184000...
97,"{'borrowHistory': [{'amount': '2000000000'}, {..."
98,{'borrowHistory': [{'amount': '360000000000000...


In [369]:
df['userid'] = 0

In [370]:
for i in range(len(df)):
    df.loc[i,'userid'] = df['user'][i]['id']

In [371]:
df['borrowHistory'] = 0

In [372]:
for i in range(len(df)):
    df['borrowHistory'][i] = df['user'][i]['borrowHistory']

C:\Users\allen\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\allen\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [373]:
df

,user,userid,borrowHistory
0,{'borrowHistory': [{'amount': '500000000000000...,0xfb5a2da7c365bfdd817bcc29a49c97b80e450f3d,"[{'amount': '500000000000000000000'}, {'amount..."
1,"{'borrowHistory': [{'amount': '1000000000'}, {...",0xecc072714365ee2fbd2e90c752c07a95641a5fe6,"[{'amount': '1000000000'}, {'amount': '1000000..."
2,{'borrowHistory': [{'amount': '900000000000000...,0x549c0421c69be943a2a60e76b19b4a801682cbd3,"[{'amount': '9000000000000000000000'}, {'amoun..."
3,"{'borrowHistory': [{'amount': '80302898'}], 'd...",0x3e20adca659b69b84591b701f6951f192943d4c3,[{'amount': '80302898'}]
4,{'borrowHistory': [{'amount': '208748722563150...,0xc4df7c5fdaed215fa559c63211407c28f91848a6,"[{'amount': '2087487225631500000000'}, {'amoun..."
...,...,...,...
95,"{'borrowHistory': [{'amount': '400000000'}, {'...",0xab292a0b800865af489fab8eb55306e3df83575f,"[{'amount': '400000000'}, {'amount': '15000000..."
96,{'borrowHistory': [{'amount': '275890249184000...,0x5104efc872d45dba3a97bf02aebe240dfc283321,"[{'amount': '2758902491840000000'}, {'amount':..."
97,"{'borrowHistory': [{'amount': '2000000000'}, {...",0xc14949d23c3620c9b08034b677dd063d27b6d3b6,"[{'amount': '2000000000'}, {'amount': '1041752..."
98,{'borrowHistory': [{'amount': '360000000000000...,0x7620fd9b67f62de89334c3ca58212c82260bc666,"[{'amount': '36000000000000000000'}, {'amount'..."


In [374]:
df['repayHistory'] = 0

In [375]:
for i in range(len(df)):
    df['repayHistory'][i] = df['user'][i]['repayHistory']

C:\Users\allen\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\allen\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [376]:
df

,user,userid,borrowHistory,repayHistory
0,{'borrowHistory': [{'amount': '500000000000000...,0xfb5a2da7c365bfdd817bcc29a49c97b80e450f3d,"[{'amount': '500000000000000000000'}, {'amount...","[{'amountAfterFee': '149000000000000000000'}, ..."
1,"{'borrowHistory': [{'amount': '1000000000'}, {...",0xecc072714365ee2fbd2e90c752c07a95641a5fe6,"[{'amount': '1000000000'}, {'amount': '1000000...","[{'amountAfterFee': '1070128726498644302479'},..."
2,{'borrowHistory': [{'amount': '900000000000000...,0x549c0421c69be943a2a60e76b19b4a801682cbd3,"[{'amount': '9000000000000000000000'}, {'amoun...",[{'amountAfterFee': '99894544142134942596615'}...
3,"{'borrowHistory': [{'amount': '80302898'}], 'd...",0x3e20adca659b69b84591b701f6951f192943d4c3,[{'amount': '80302898'}],[{'amountAfterFee': '80976429'}]
4,{'borrowHistory': [{'amount': '208748722563150...,0xc4df7c5fdaed215fa559c63211407c28f91848a6,"[{'amount': '2087487225631500000000'}, {'amoun...","[{'amountAfterFee': '3721250430594437767510'},..."
...,...,...,...,...
95,"{'borrowHistory': [{'amount': '400000000'}, {'...",0xab292a0b800865af489fab8eb55306e3df83575f,"[{'amount': '400000000'}, {'amount': '15000000...","[{'amountAfterFee': '2484337807'}, {'amountAft..."
96,{'borrowHistory': [{'amount': '275890249184000...,0x5104efc872d45dba3a97bf02aebe240dfc283321,"[{'amount': '2758902491840000000'}, {'amount':...","[{'amountAfterFee': '1319374971'}, {'amountAft..."
97,"{'borrowHistory': [{'amount': '2000000000'}, {...",0xc14949d23c3620c9b08034b677dd063d27b6d3b6,"[{'amount': '2000000000'}, {'amount': '1041752...","[{'amountAfterFee': '661391158'}, {'amountAfte..."
98,{'borrowHistory': [{'amount': '360000000000000...,0x7620fd9b67f62de89334c3ca58212c82260bc666,"[{'amount': '36000000000000000000'}, {'amount'...","[{'amountAfterFee': '689999928128040747108'}, ..."


In [377]:
df['liquidationCallHistory'] = '0'

In [378]:
for i in range(len(df)):
    if np.array(df['user'][i]['liquidationCallHistory']).size > 0:
        df['liquidationCallHistory'][i] = df['user'][i]['liquidationCallHistory']

In [379]:
df

,user,userid,borrowHistory,repayHistory,liquidationCallHistory
0,{'borrowHistory': [{'amount': '500000000000000...,0xfb5a2da7c365bfdd817bcc29a49c97b80e450f3d,"[{'amount': '500000000000000000000'}, {'amount...","[{'amountAfterFee': '149000000000000000000'}, ...",0
1,"{'borrowHistory': [{'amount': '1000000000'}, {...",0xecc072714365ee2fbd2e90c752c07a95641a5fe6,"[{'amount': '1000000000'}, {'amount': '1000000...","[{'amountAfterFee': '1070128726498644302479'},...",[{'collateralAmount': '547024554576355662089'}...
2,{'borrowHistory': [{'amount': '900000000000000...,0x549c0421c69be943a2a60e76b19b4a801682cbd3,"[{'amount': '9000000000000000000000'}, {'amoun...",[{'amountAfterFee': '99894544142134942596615'}...,0
3,"{'borrowHistory': [{'amount': '80302898'}], 'd...",0x3e20adca659b69b84591b701f6951f192943d4c3,[{'amount': '80302898'}],[{'amountAfterFee': '80976429'}],0
4,{'borrowHistory': [{'amount': '208748722563150...,0xc4df7c5fdaed215fa559c63211407c28f91848a6,"[{'amount': '2087487225631500000000'}, {'amoun...","[{'amountAfterFee': '3721250430594437767510'},...",0
...,...,...,...,...,...
95,"{'borrowHistory': [{'amount': '400000000'}, {'...",0xab292a0b800865af489fab8eb55306e3df83575f,"[{'amount': '400000000'}, {'amount': '15000000...","[{'amountAfterFee': '2484337807'}, {'amountAft...",0
96,{'borrowHistory': [{'amount': '275890249184000...,0x5104efc872d45dba3a97bf02aebe240dfc283321,"[{'amount': '2758902491840000000'}, {'amount':...","[{'amountAfterFee': '1319374971'}, {'amountAft...",[{'collateralAmount': '1636628455949431080364'...
97,"{'borrowHistory': [{'amount': '2000000000'}, {...",0xc14949d23c3620c9b08034b677dd063d27b6d3b6,"[{'amount': '2000000000'}, {'amount': '1041752...","[{'amountAfterFee': '661391158'}, {'amountAfte...",0
98,{'borrowHistory': [{'amount': '360000000000000...,0x7620fd9b67f62de89334c3ca58212c82260bc666,"[{'amount': '36000000000000000000'}, {'amount'...","[{'amountAfterFee': '689999928128040747108'}, ...",0


I am creating count columns for the number of borrows, repayments, and liquidations that a user has encountered. This may be useful information for the credit scoring metric.

In [380]:
df['borrowCount'] = 0

In [381]:
for i in range(len(df)):
    df.loc[i,'borrowCount'] = len(df['borrowHistory'][i])

In [382]:
df

,user,userid,borrowHistory,repayHistory,liquidationCallHistory,borrowCount
0,{'borrowHistory': [{'amount': '500000000000000...,0xfb5a2da7c365bfdd817bcc29a49c97b80e450f3d,"[{'amount': '500000000000000000000'}, {'amount...","[{'amountAfterFee': '149000000000000000000'}, ...",0,6
1,"{'borrowHistory': [{'amount': '1000000000'}, {...",0xecc072714365ee2fbd2e90c752c07a95641a5fe6,"[{'amount': '1000000000'}, {'amount': '1000000...","[{'amountAfterFee': '1070128726498644302479'},...",[{'collateralAmount': '547024554576355662089'}...,7
2,{'borrowHistory': [{'amount': '900000000000000...,0x549c0421c69be943a2a60e76b19b4a801682cbd3,"[{'amount': '9000000000000000000000'}, {'amoun...",[{'amountAfterFee': '99894544142134942596615'}...,0,73
3,"{'borrowHistory': [{'amount': '80302898'}], 'd...",0x3e20adca659b69b84591b701f6951f192943d4c3,[{'amount': '80302898'}],[{'amountAfterFee': '80976429'}],0,1
4,{'borrowHistory': [{'amount': '208748722563150...,0xc4df7c5fdaed215fa559c63211407c28f91848a6,"[{'amount': '2087487225631500000000'}, {'amoun...","[{'amountAfterFee': '3721250430594437767510'},...",0,4
...,...,...,...,...,...,...
95,"{'borrowHistory': [{'amount': '400000000'}, {'...",0xab292a0b800865af489fab8eb55306e3df83575f,"[{'amount': '400000000'}, {'amount': '15000000...","[{'amountAfterFee': '2484337807'}, {'amountAft...",0,13
96,{'borrowHistory': [{'amount': '275890249184000...,0x5104efc872d45dba3a97bf02aebe240dfc283321,"[{'amount': '2758902491840000000'}, {'amount':...","[{'amountAfterFee': '1319374971'}, {'amountAft...",[{'collateralAmount': '1636628455949431080364'...,100
97,"{'borrowHistory': [{'amount': '2000000000'}, {...",0xc14949d23c3620c9b08034b677dd063d27b6d3b6,"[{'amount': '2000000000'}, {'amount': '1041752...","[{'amountAfterFee': '661391158'}, {'amountAfte...",0,21
98,{'borrowHistory': [{'amount': '360000000000000...,0x7620fd9b67f62de89334c3ca58212c82260bc666,"[{'amount': '36000000000000000000'}, {'amount'...","[{'amountAfterFee': '689999928128040747108'}, ...",0,33


In [383]:
df['repayCount'] = 0

In [384]:
for i in range(len(df)):
    df.loc[i,'repayCount'] = len(df['repayHistory'][i])

In [385]:
df

,user,userid,borrowHistory,repayHistory,liquidationCallHistory,borrowCount,repayCount
0,{'borrowHistory': [{'amount': '500000000000000...,0xfb5a2da7c365bfdd817bcc29a49c97b80e450f3d,"[{'amount': '500000000000000000000'}, {'amount...","[{'amountAfterFee': '149000000000000000000'}, ...",0,6,7
1,"{'borrowHistory': [{'amount': '1000000000'}, {...",0xecc072714365ee2fbd2e90c752c07a95641a5fe6,"[{'amount': '1000000000'}, {'amount': '1000000...","[{'amountAfterFee': '1070128726498644302479'},...",[{'collateralAmount': '547024554576355662089'}...,7,3
2,{'borrowHistory': [{'amount': '900000000000000...,0x549c0421c69be943a2a60e76b19b4a801682cbd3,"[{'amount': '9000000000000000000000'}, {'amoun...",[{'amountAfterFee': '99894544142134942596615'}...,0,73,43
3,"{'borrowHistory': [{'amount': '80302898'}], 'd...",0x3e20adca659b69b84591b701f6951f192943d4c3,[{'amount': '80302898'}],[{'amountAfterFee': '80976429'}],0,1,1
4,{'borrowHistory': [{'amount': '208748722563150...,0xc4df7c5fdaed215fa559c63211407c28f91848a6,"[{'amount': '2087487225631500000000'}, {'amoun...","[{'amountAfterFee': '3721250430594437767510'},...",0,4,4
...,...,...,...,...,...,...,...
95,"{'borrowHistory': [{'amount': '400000000'}, {'...",0xab292a0b800865af489fab8eb55306e3df83575f,"[{'amount': '400000000'}, {'amount': '15000000...","[{'amountAfterFee': '2484337807'}, {'amountAft...",0,13,11
96,{'borrowHistory': [{'amount': '275890249184000...,0x5104efc872d45dba3a97bf02aebe240dfc283321,"[{'amount': '2758902491840000000'}, {'amount':...","[{'amountAfterFee': '1319374971'}, {'amountAft...",[{'collateralAmount': '1636628455949431080364'...,100,100
97,"{'borrowHistory': [{'amount': '2000000000'}, {...",0xc14949d23c3620c9b08034b677dd063d27b6d3b6,"[{'amount': '2000000000'}, {'amount': '1041752...","[{'amountAfterFee': '661391158'}, {'amountAfte...",0,21,10
98,{'borrowHistory': [{'amount': '360000000000000...,0x7620fd9b67f62de89334c3ca58212c82260bc666,"[{'amount': '36000000000000000000'}, {'amount'...","[{'amountAfterFee': '689999928128040747108'}, ...",0,33,25


In [386]:
df['liquidationCount'] = 0

In [387]:
for i in range(len(df)):
    if df['liquidationCallHistory'][i] != '0':
        df.loc[i,'liquidationCount'] = len(df['liquidationCallHistory'][i])

In [388]:
df

,user,userid,borrowHistory,repayHistory,liquidationCallHistory,borrowCount,repayCount,liquidationCount
0,{'borrowHistory': [{'amount': '500000000000000...,0xfb5a2da7c365bfdd817bcc29a49c97b80e450f3d,"[{'amount': '500000000000000000000'}, {'amount...","[{'amountAfterFee': '149000000000000000000'}, ...",0,6,7,0
1,"{'borrowHistory': [{'amount': '1000000000'}, {...",0xecc072714365ee2fbd2e90c752c07a95641a5fe6,"[{'amount': '1000000000'}, {'amount': '1000000...","[{'amountAfterFee': '1070128726498644302479'},...",[{'collateralAmount': '547024554576355662089'}...,7,3,3
2,{'borrowHistory': [{'amount': '900000000000000...,0x549c0421c69be943a2a60e76b19b4a801682cbd3,"[{'amount': '9000000000000000000000'}, {'amoun...",[{'amountAfterFee': '99894544142134942596615'}...,0,73,43,0
3,"{'borrowHistory': [{'amount': '80302898'}], 'd...",0x3e20adca659b69b84591b701f6951f192943d4c3,[{'amount': '80302898'}],[{'amountAfterFee': '80976429'}],0,1,1,0
4,{'borrowHistory': [{'amount': '208748722563150...,0xc4df7c5fdaed215fa559c63211407c28f91848a6,"[{'amount': '2087487225631500000000'}, {'amoun...","[{'amountAfterFee': '3721250430594437767510'},...",0,4,4,0
...,...,...,...,...,...,...,...,...
95,"{'borrowHistory': [{'amount': '400000000'}, {'...",0xab292a0b800865af489fab8eb55306e3df83575f,"[{'amount': '400000000'}, {'amount': '15000000...","[{'amountAfterFee': '2484337807'}, {'amountAft...",0,13,11,0
96,{'borrowHistory': [{'amount': '275890249184000...,0x5104efc872d45dba3a97bf02aebe240dfc283321,"[{'amount': '2758902491840000000'}, {'amount':...","[{'amountAfterFee': '1319374971'}, {'amountAft...",[{'collateralAmount': '1636628455949431080364'...,100,100,2
97,"{'borrowHistory': [{'amount': '2000000000'}, {...",0xc14949d23c3620c9b08034b677dd063d27b6d3b6,"[{'amount': '2000000000'}, {'amount': '1041752...","[{'amountAfterFee': '661391158'}, {'amountAfte...",0,21,10,0
98,{'borrowHistory': [{'amount': '360000000000000...,0x7620fd9b67f62de89334c3ca58212c82260bc666,"[{'amount': '36000000000000000000'}, {'amount'...","[{'amountAfterFee': '689999928128040747108'}, ...",0,33,25,0


I created a scoring metric that is modelled off the structure of a sigmoid function. The purpose of this is to constrain the score between 0 and 1. However we want to scale it up three decimal places to model the scale and structure of a credit score or Spectral's MACRO score. The structure of a sigmoid function is 1/(1+e^-x) where x has the ability of any number on a small or large scale. X is going to take into account the three metrics that we extracted from AAVE's API which is a user's borrowCount, repayCount, and liquidationCount. In an ideal world, borrowCount is exactly equal to repayCount which a borrower pays back all of his loans leading to an equation of 1/(1+e^-1) which is around 0.73. Similar to credit score, a good score lies around 0.7 * 1000 which is 700. In terms of the real world, the scalability is similar to our current standards of credit score and FICO score. In unique cases where the repayCount is higher than borrowCount, the credit score will go above 730 to 800, and in extreme cases and outliers it will go above that. It seems that in most of these entries the repayCount is lower than the borrowCount which makes sense. The x in the case of the sigmoid function can be modelled by a ratio of repayCount to borrowCount. 

Taking into account liquidationCounts, liquidations should penalize whatever score that we had derived from the sigmoid function. We can't subtract a flat number or a large chunk of the score per liquidation because that would be too punishing. Instead, we penalize the score per liquidationCount in respect to the number of borrowCounts. This can be an addition into the x variable of the sigmoid function. The new formula is modelled by 1000/(1+e^((-repayCount+liquidationCount)/borrowCount)). In this model, if there are more liquidationCount than repayCount, then the score is going to be severely punishing as the score dips below 500 just as e's power becomes positive.

${creditscore} = \Large\frac {1}{1+e^\frac{-repayCount+liquidationCount}{borrowCount}} $

In [389]:
df['creditscore'] = 0

In [390]:
for i in range(len(df)):
    df.loc[i,'creditscore'] = 1000/(1+np.exp((-df.repayCount[i]+df.liquidationCount[i])/df.borrowCount[i]))

In [391]:
df.head()

,user,userid,borrowHistory,repayHistory,liquidationCallHistory,borrowCount,repayCount,liquidationCount,creditscore
0,{'borrowHistory': [{'amount': '500000000000000...,0xfb5a2da7c365bfdd817bcc29a49c97b80e450f3d,"[{'amount': '500000000000000000000'}, {'amount...","[{'amountAfterFee': '149000000000000000000'}, ...",0,6,7,0,762.541972
1,"{'borrowHistory': [{'amount': '1000000000'}, {...",0xecc072714365ee2fbd2e90c752c07a95641a5fe6,"[{'amount': '1000000000'}, {'amount': '1000000...","[{'amountAfterFee': '1070128726498644302479'},...",[{'collateralAmount': '547024554576355662089'}...,7,3,3,500.000000
2,{'borrowHistory': [{'amount': '900000000000000...,0x549c0421c69be943a2a60e76b19b4a801682cbd3,"[{'amount': '9000000000000000000000'}, {'amoun...",[{'amountAfterFee': '99894544142134942596615'}...,0,73,43,0,643.145098
3,"{'borrowHistory': [{'amount': '80302898'}], 'd...",0x3e20adca659b69b84591b701f6951f192943d4c3,[{'amount': '80302898'}],[{'amountAfterFee': '80976429'}],0,1,1,0,731.058579
4,{'borrowHistory': [{'amount': '208748722563150...,0xc4df7c5fdaed215fa559c63211407c28f91848a6,"[{'amount': '2087487225631500000000'}, {'amoun...","[{'amountAfterFee': '3721250430594437767510'},...",0,4,4,0,731.058579


In [392]:
df.creditscore.unique()

array([762.54197166, 500.        , 643.14509833, 731.05857863,
       670.03278287, 758.20382759, 740.77489918, 536.96944503,
       707.30957093, 783.42090423, 549.83399731, 802.18388856,
       982.01379004, 719.33723773, 689.24889694, 671.94189569,
       710.94950263, 753.55200924, 610.63923395, 705.78502784,
       486.11468225, 594.98579352, 810.69699831, 584.19052294,
       622.4593312 , 625.06690871, 743.96249132, 582.57020646,
       702.66065434, 547.47559513, 658.6182792 , 670.32987736,
       676.99585624, 717.28511248, 694.539502  , 544.52460417,
       647.64320532, 575.18313194, 785.83498304, 657.50018166,
       689.97448113, 639.09274516, 513.15485821, 641.06740633,
       609.31754184, 685.94945501, 535.65367083, 679.17869918,
       524.97918748, 880.79707798, 552.43804487, 712.38111293,
       631.45445985, 420.67574785, 697.05928397, 736.71916799,
       645.65630623, 660.75636877, 670.86970061, 619.95605415,
       791.39147267, 526.01814468, 472.25076495, 692.64

The scoring formula can definitely take into account other covariates and can be further improved with additional metrics. Since this is a very underdeveloped prototype of a credit scoring method, it only takes three variables. If we had blockchain data with already existing scores from a previous scoring metric but no knowledge of the scoring metrics, we could model and predict these continuous scores using machine learning specifically linear regression. However since we are creating an entirely new scoring metric, we are going off of heuristics. We can use off-chain data to improve the model by scanning for outliers in calculating the credit score using this given formula. There we would perform root-cause-analysis in order to determine what is causing the outliers and extremities (e.g. if we have multiple scores that are way below 500 or above 900) and to adjust our model accordingly.